In [12]:
import pandas as pd
movies=pd.read_csv('/movies.csv')
ratings=pd.read_csv('/ratings.csv')
movies["year"]=movies["title"].str.extract(r'\((\d{4})\)')
merged_df = pd.merge(movies, ratings, on='movieId')
def assign_rating(row):
    if row['rating'] < 3:
        return [1, 0, 0]
    elif row['rating'] == 3:
        return [0, 1, 0]
    else:
        return [0, 0, 1]

merged_df[['rating_low', 'rating_medium', 'rating_high']] = merged_df.apply(assign_rating, axis=1, result_type='expand')

def assign_year(row):
    try:
        if int(row["year"])<2000:
            return [1,0]
        else:
            return [0,1]
    except:
        return [0,0]

merged_df[["year_old","year_new"]] = merged_df.apply(assign_year,axis=1, result_type='expand')

genres_df = merged_df['genres'].str.get_dummies(sep='|')
merged_df = pd.concat([merged_df, genres_df], axis=1)
merged_df

,movieId,title,genres,year,userId,rating,timestamp,rating_low,rating_medium,rating_high,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,1,4.0,964982703,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,5,4.0,847434962,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,7,4.5,1106635946,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,15,2.5,1510577970,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,17,4.5,1305696483,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100831,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,2017,184,4.0,1537109082,0,0,1,...,0,0,0,0,0,0,0,0,0,0
100832,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,2017,184,3.5,1537109545,0,0,1,...,0,0,0,0,0,0,0,0,0,0
100833,193585,Flint (2017),Drama,2017,184,3.5,1537109805,0,0,1,...,0,0,0,0,0,0,0,0,0,0
100834,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,2018,184,3.5,1537110021,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [13]:
merged_df['datetime'] = pd.to_datetime(merged_df['timestamp'], unit='s')
merged_df['review_year'] = merged_df['datetime'].dt.year
merged_df['month'] = merged_df['datetime'].dt.month
merged_df['day'] = merged_df['datetime'].dt.day
merged_df['day_of_the_week'] = merged_df['datetime'].dt.dayofweek
merged_df['hour'] = merged_df['datetime'].dt.hour

In [14]:
merged_df=merged_df.drop(columns=['datetime','movieId','title','genres', 'year', 'userId', 'rating', 'timestamp', '(no genres listed)'])

In [18]:
def assign_year(row):
    try:
        if int(row["review_year"])<2000:
            return [1,0]
        else:
            return [0,1]
    except:
        return [0,0]

def assign_month(row):
    months=[0 for i in range(12)] 
    months[row["month"]-1]=1
    return months

def assign_day(row):
    days=[0 for i in range(7)] 
    days[row["day_of_the_week"]-1]=1
    return days

def assign_time_day(row):
    if row["hour"]>5 and row["hour"]<12:
        return [1,0,0]
    elif row["hour"]>=12 and row["hour"]<=22:
        return [0,1,0]
    else:
        return [0,0,1]

month_names = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

merged_df[["old_review","recent_review"]] = merged_df.apply(assign_year,axis=1, result_type='expand')
merged_df[['reviewed_'+month_names[i] for i in range(len(month_names))]] = merged_df.apply(assign_month,axis=1, result_type='expand')
merged_df[['reviewed_'+days_of_week[i] for i in range(len(days_of_week))]] = merged_df.apply(assign_day, axis=1, result_type="expand")
merged_df[["reviewed_morning", "reviewed_afternoon", "reviewed_night"]] = merged_df.apply(assign_time_day,axis=1, result_type="expand")
merged_df

,rating_low,rating_medium,rating_high,year_old,year_new,Action,Adventure,Animation,Children,Comedy,...,reviewed_Monday,reviewed_Tuesday,reviewed_Wednesday,reviewed_Thursday,reviewed_Friday,reviewed_Saturday,reviewed_Sunday,reviewed_morning,reviewed_afternoon,reviewed_night
0,0,0,1,1,0,0,1,1,1,1,...,0,0,0,0,0,1,0,0,1,0
1,0,0,1,1,0,0,1,1,1,1,...,0,0,0,1,0,0,0,1,0,0
2,0,0,1,1,0,0,1,1,1,1,...,1,0,0,0,0,0,0,1,0,0
3,1,0,0,1,0,0,1,1,1,1,...,0,0,0,0,0,0,1,0,1,0
4,0,0,1,1,0,0,1,1,1,1,...,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100831,0,0,1,0,1,1,0,1,0,1,...,0,0,0,0,0,1,0,0,1,0
100832,0,0,1,0,1,0,0,1,0,1,...,0,0,0,0,0,1,0,0,1,0
100833,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
100834,0,0,1,0,1,1,0,1,0,0,...,0,0,0,0,0,1,0,0,1,0


In [20]:
merged_df=merged_df.drop(columns=['review_year', 'month', 'day', 'day_of_the_week', 'hour'])

In [22]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

frequent_itemsets = apriori(merged_df, min_support=0.01, use_colnames=True)

association_rules(frequent_itemsets, 
                  metric='confidence', 
                  min_threshold=0.2)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(rating_low),(year_old),0.189149,0.610546,0.109802,0.580506,0.950799,-0.005682,0.928391
1,(year_new),(rating_low),0.389276,0.189149,0.079317,0.203755,1.077222,0.005686,1.018344
2,(rating_low),(year_new),0.189149,0.389276,0.079317,0.419336,1.077222,0.005686,1.051769
3,(Action),(rating_low),0.303810,0.189149,0.062031,0.204178,1.079459,0.004566,1.018886
4,(rating_low),(Action),0.189149,0.303810,0.062031,0.327951,1.079459,0.004566,1.035921
...,...,...,...,...,...,...,...,...,...
28869,"(Comedy, Animation)","(recent_review, Children, Adventure, rating_hi...",0.037923,0.016502,0.010036,0.264644,16.037066,0.009410,1.337445
28870,"(year_new, Children)","(recent_review, Animation, Adventure, rating_h...",0.032816,0.016730,0.010036,0.305833,18.280340,0.009487,1.416474
28871,"(Adventure, Animation)","(recent_review, Children, year_new, rating_hig...",0.040938,0.014757,0.010036,0.245155,16.613208,0.009432,1.305226
28872,"(rating_high, Animation)","(recent_review, Children, year_new, Adventure,...",0.046670,0.015778,0.010036,0.215045,13.629315,0.009300,1.253857
